<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/Retrieve-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [210]:
target = new_client.target
target_query = target.search('A375')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,A375-SM,18.0,False,CHEMBL4513121,[],CELL-LINE,9606
1,[],Homo sapiens,A-375,17.0,False,CHEMBL613859,[],CELL-LINE,9606


In [211]:
selected_target = targets.target_chembl_id[1]
selected_target

'CHEMBL613859'

In [218]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [219]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,153293,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,0.004
1,None,156918,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,2.0
2,None,158163,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,1.0
3,None,158167,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,0.07
4,None,159375,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,None,24516301,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,2.089
4025,None,24516302,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,2.472
4026,None,24516303,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,50.0
4027,None,24516304,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,A-375,9606,None,None,IC50,uM,UO_0000065,None,100.0


In [220]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
df = df[df['standard_value'].notna()]
df = df[df['canonical_smiles'].notna()]

len(df)

3134

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [221]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 20000 <= i <= 100000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df

<ipython-input-221-d453deb383ef>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,153293,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,4.0,Yes
1,None,156918,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,2000.0,IN
2,None,158163,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,1000.0,IN
3,None,158167,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,70.0,Yes
4,None,159375,[],CHEMBL621042,In vitro antitumor activity against A375 (mela...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,130.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4022,None,24516299,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,1100.0,IN
4023,None,24516300,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,2527.0,IN
4024,None,24516301,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,2089.0,IN
4025,None,24516302,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5057275,Cytotoxicity against human A-375 cells assesse...,F,None,None,BAO_0000190,BAO_0000219,...,A-375,9606,None,None,IC50,uM,UO_0000065,None,2472.0,IN


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [222]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

1547

In [223]:
df.to_csv('bioactivity_data_A375_IC50_nM.csv', index=False)   